In [6]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Name 1: Alex Matsunami
- Name 2: Andrew Clark
- Name 3: Cory Nichols

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [7]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.121435315436 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 
- What does each instance represent? 
- How many classes are in the dataset and what does each class represent?
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?

___
Enter your answer here:

There are 11,314 instances in the dataset, each representing a partition of a newsgroup relating to a different topic. Each row represents the total words across all articles. 130,000 + words, frequency of each term is taken and normalize how often we expect word to occur as a whole (TFIDF). Divides the number of times we see that word by number of times we expect to see the word. 

There are 20 classes in the dataset representing each newsgroup genre like sports or computer hardware. We want to classify these articles into their respective genres. 

Documents written in the past week would likely NOT generalize well given some of the topics are hardware related, which changes dramatically over the years, relgious topics, which also change dynamically. It would work well on more static genres like foreign policy, however. 

Given that the data is only 0.12% non-zero, this is an extremely SPARSE matrix





___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [8]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

a_bin, b_bin, c_bin = a,b,c
for i in [a,b,c]:
    i[i>0] = 1

# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean(a,c), 'bc:',euclidean(b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:', cosine(b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard(a_bin,b_bin), 
      'ac:', jaccard(a_bin,c_bin), 'bc:', jaccard(b_bin, c_bin))

print('\n\nThe most appropriate distance is...')
print('Cosine similarity performs well on large, sparse, document based datasets, Cosine would be our best bet here\
        because we are given word frequency percentages. Cosine handles non-binary data and will allow us to use\
        sparse data matrices for our distance measurements, because we are not looking at 0-0 matches')

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 14.7309198627 ac: 15.2315462117 bc: 16.7032930885
Cosine Distance
 ab: 0.787155605009 ac: 0.77389303337 bc: 0.832440437912
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.882113821138 ac: 0.875471698113 bc: 0.908794788274


The most appropriate distance is...
Cosine similarity performs well on large, sparse, document based datasets, Cosine would be our best bet here        because we are given word frequency percentages. Cosine handles non-binary data and will allow us to use        sparse data matrices for our distance measurements, because we are not looking at 0-0 matches


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [11]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets

cv = StratifiedShuffleSplit(ds.target, n_iter = 1, test_size = 0.5, train_size=0.5)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K
#K = 5 
#clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')
#clf.fit(X_train, y_train)
#acc = accuracy_score(clf.predict(X_test), y_test) # overwrite this with the actual accuracy



#=====================================
for i in [1,2,3,4,5]:
    clf = KNeighborsClassifier(n_neighbors=i, weights='uniform', metric='euclidean')
    clf.fit(X_train, y_train)
    acc = accuracy_score(clf.predict(X_test), y_test) # overwrite this with the actual accuracy
    print('Accuracy of classifier with %d neighbors is: %.2f'%(i,acc))


Accuracy of classifier with 1 neighbors is: 0.61
Accuracy of classifier with 2 neighbors is: 0.56
Accuracy of classifier with 3 neighbors is: 0.54
Accuracy of classifier with 4 neighbors is: 0.53
Accuracy of classifier with 5 neighbors is: 0.52


The best accuracy is 61% with 1 neighbor.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:

No, it does not, as the KDTree will continually add blocks or regions to check in order to find nearest neighbors the more attributes we add to the data set. Relationship of regions to scan is exponential with the amount of attributes in our data set. We would need to search through more and we are lazy, so we don't want to do that.
___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [15]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)

for i in ['euclidean','manhattan','cosine','cityblock']:
    clf = NearestCentroid(metric=i)
    clf.fit(X_train, y_train)
    acc = accuracy_score(clf.predict(X_test), y_test) # overwrite this with the actual accuracy
    print('Accuracy of NearestCentroid classifier with %s distance is: %.2f'%(i, acc))

# fill in your code here

print('The best distance metric is: Cosine')

Accuracy of NearestCentroid classifier with euclidean distance is: 0.40
Accuracy of NearestCentroid classifier with manhattan distance is: 0.17
Accuracy of NearestCentroid classifier with cosine distance is: 0.48
Accuracy of NearestCentroid classifier with cityblock distance is: 0.34
The best distance metric is: Cosine


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Enter you answer here:

We would have 20 priors, which is the number of classes

We would also have 2,602,140 conditional probabilities, way too many for this refurbished mac to calculate


In [16]:
# Use this space for any calculations you might want to do
print 'number of conditional probabilities', 20*130107

2602140

___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:

If the data is mostly continuous, we would use Gaussian, especially given our sample size and tenants of the CLT. 

If it's sparse and non-binary, we would use a Multinomial Naive Bayes classifier, because our dataset is non-binary. We would lose tf-idf information if we were to binarize our dataset and likely impact accuracy of the classifier. Cannot use a gaussian because of sparsity of data. Random result would occur.

___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

# alpha is smoothing of the dataset, allows us to account for conditional probabilities that are non-existent or zero
# which would take our posterior probability to zero


# binarize sets threshold for binarization, adds additonal penalty if 
# class doesnt occur

# higher = more smoothing
# enter poor man's grid search
for i in [0.0,0.25,0.5,0.75,1,2,3,4,5]:
    clf_mnb = MultinomialNB(alpha=i)
    clf_mnb.fit(X_train, y_train)
    acc = accuracy_score(clf_mnb.predict(X_test), y_test) # overwrite this with the actual accuracy
    print('Accuracy of MultiNomialNB classifier with %s alpha is: %.2f'%(i, acc))
    
for i in [0.0,0.25,0.5,0.75,1.0,2.0,3.0]:
    for j in [0.0,0.25,0.5,0.75]:
        clf_bnb = BernoulliNB(alpha=i, binarize=j)
        clf_bnb.fit(X_train, y_train)
        acc = accuracy_score(clf_bnb.predict(X_test), y_test) # overwrite this with the actual accuracy
        print('Accuracy of BernoulliNB classifier with %s alpha and %s binarization is: %.2f'%(i,j,acc))
        
print('''\n These classifiers are so fast because they are eager learners and the prior probability 
            values are stored ahead of time. Further, it assumes independence of attributes, 
            so our attributes do not need conditional probs calculated as well. We can iterate 
            through all of the new examples with our already calculated prior probabilities''')
print('''\nThe alpha values control the smoothing, or the ability to work with missing data. 
           If alpha > 0, we get lidstone smoothing, and with alpha = 1, we get laplace smoothing (adding 1)''')


Accuracy of MultiNomialNB classifier with 0.0 alpha is: 0.11
Accuracy of MultiNomialNB classifier with 0.25 alpha is: 0.80
Accuracy of MultiNomialNB classifier with 0.5 alpha is: 0.77
Accuracy of MultiNomialNB classifier with 0.75 alpha is: 0.74
Accuracy of MultiNomialNB classifier with 1 alpha is: 0.71
Accuracy of MultiNomialNB classifier with 2 alpha is: 0.63
Accuracy of MultiNomialNB classifier with 3 alpha is: 0.56
Accuracy of MultiNomialNB classifier with 4 alpha is: 0.50
Accuracy of MultiNomialNB classifier with 5 alpha is: 0.46
Accuracy of BernoulliNB classifier with 0.0 alpha and 0.0 binarization is: 0.04


________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.